In [152]:
import csv
from math import sqrt
from math import log2
from collections import Counter

In [5]:
# 讀取 txt 檔案
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        data = [row for row in reader]
    return data
# 讀取資料
data = load_data('glass.txt')

In [6]:
# 定義欄位名稱
columns = ["Id","RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
# 轉換成字典
df = [dict(zip(columns, row)) for row in data]

In [110]:
def table_to_column_dict(data, columns, convert_numeric=True):
    df_dict = {col: [] for col in columns}
    
    for row in data:
        for col, val in zip(columns, row):
            if convert_numeric:
                try:
                    val = float(val)
                except ValueError:
                    pass  # 若轉不了 float，就保持原樣（例如 Id）
            df_dict[col].append(val)
    
    return df_dict
df = table_to_column_dict(data,columns)

In [127]:
X = columns.copy()
X.remove("Id")
X.remove("class")
y = 'class'
X

['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']

In [132]:
# 計算 feature entropy
def entropy(df,feature):  
    att_value = df[feature]
    value_count = Counter(att_value)          # 計算所有可能值的個數
    total = len(att_value)
    prob = [count / total for key,count in value_count.items()]  # 計算每個 attribute_value 的機率
    return -sum(p * log2(p) for p in prob)
# 計算特徵 X、Y 間的 Mutual Information
def mutual_information(df,X, Y):
    X_list = df[X]
    Y_list = df[Y]

    # 計算 X 和 Y 的熵
    H_X = entropy(df ,X)
    H_Y = entropy(df ,Y)
    # 計算 X 和 Y 的聯合機率
    joint_pairs = list(zip(X_list, Y_list))
    joint_counts = Counter(joint_pairs)
    total = len(X_list)
    joint_prob = [count / total for key,count in joint_counts.items()]
    print(joint_prob)
    H_X_Y = -sum(p * log2(p) for p in joint_prob)
    return H_X + H_Y - H_X_Y
    
# 計算特徵 X、Y 的 symmetric uncertainty
def cal_su(df,X,Y):
    H_X = entropy(df,X)
    H_Y = entropy(df,Y)
    return 2 * (mutual_information(df,X,Y) / (H_X + H_Y))

# 計算選取的特徵子集對於類別預測的 Goodness
def Goodness(df,feature_subset,label):
    su_X_C = 0
    sum_su_X_Y = 0  
    # 計算 feature_subset 內所有特徵對於類別值的 Symmetric uncertainty
    su_X_C = sum(cal_su(df,X,label) for X in feature_subset)
    
    # 計算 feature_subset 內所有兩兩特徵間的 Symmetric uncertainty
    for feature_i in feature_subset:
        for feature_j in feature_subset:
            sum_su_X_Y += cal_su(df,feature_i,feature_j)
    return su_X_C / sqrt(sum_su_X_Y)

# X 代表資料集的特徵集合，y 則是類別值
# forward_selection 函式會決定最後 Goodness 最優的 feature subset

def forward_selection(df, X, y):
    select_features = []    # 儲存每回最優的 feature subset
    best_score = 0.0        # 每一列 feature subset 中最優的 Goodness
    remaining_features = X.copy()  # 還未被選定的 features
    # 持續檢查直到沒有可以選擇的 feature
    i = 1   # 紀錄 foward selection 到第幾輪 
    while(len(remaining_features) > 0):
        scores = []  # 儲存這一列中每個 subset 的 Goodness
        for feature in remaining_features:
            # temp_features 暫存此次循環的特徵組合 => 上回以選取好的最佳組合 select_features + 這回新選入的一個 feature
            temp_features = select_features + [feature]
            score = Goodness(df,temp_features,y)
            # (目前的特徵組合, 新選進來的特徵, 此特徵組合的 Goodness)
            scores.append((temp_features,feature,score))

        # 依照 Goodness 排序
        scores.sort(key=lambda x: x[2], reverse = True)  
        best_new_score = float(scores[0][2])   # 這一輪特徵組合中最優的 Goodness

        if(best_new_score > best_score):
            best_score = best_new_score
            select_features = scores[0][0]  # 更新成 Goodness 最優的 subset
            if scores[0][1] in remaining_features:
                remaining_features.remove(scores[0][1])  # 移除新選特徵
            print(f"Pass{i}: best_feature_subset = {select_features} , Goodness = {best_score}")
            i += 1
        # 此輪中所有 feature_subset 的表現皆不如上一輪，Stop
        else:
            break
    print(f"\nFinal select features: {select_features}, Goodness = {best_score}")
    return select_features,best_score

def backward_selection(df, X, y):
    select_features = X     # 一開始是選擇所有 attributes  
    best_score = 0.0        # 每一列 feature subset 中最優的 Goodness
    i = 1
    # 持續檢查到選擇的 feature 只剩下一個
    while(len(select_features) > 1):
        scores = []  # 儲存這一列中每個 subset 的 Goodness
        for feature in select_features:
            temp_features = select_features.copy()
            # 每次移除一個 feature
            temp_features.remove(feature)
            score = Goodness(df,temp_features,y)
            # (目前的特徵組合, 移除的特徵, 此特徵組合的 Goodness)
            scores.append((temp_features,feature,score))

        scores.sort(key = lambda x: x[2], reverse = True)  
        best_new_score = float(scores[0][2])   # 這一輪特徵組合中最優的 Goodness
        # 代表移除該特徵後 Goodness 更好
        if(best_new_score > best_score):
            best_score = best_new_score
            select_features = scores[0][0]  # 更新成 Goodness 最優的 subset
            if scores[0][1] in select_features:
                select_features.remove(scores[0][1])  # 移除特徵
            print(f"Pass{i}: best_feature_subset = {select_features} , Goodness = {best_score}")
            print(f"\nremove feature: {scores[0][1]}\n")
            i += 1
        # 此輪中所有 feature_subset 的表現皆不如上一輪，Stop
        else:
            break
    print(f"\nFinal select features: {select_features}, Goodness = {best_score}")
    return select_features,best_score

### Equal Width

In [128]:
def equal_width(df,feature,bin_num):
    # 計算每組 bin 區間
    att_value = df[feature]
    max_value = float(max(att_value))
    min_value = float(min(att_value))
    # 計算每組區間寬度
    width = (max_value - min_value) / bin_num

    bins = []  # 儲存每組區間範圍
    # 計算每組區間數值範圍
    for i in range(bin_num):
        start = min_value + i * width
        end = start + width
        bins.append((start,end)) 
    print(f'{feature} with equal width discretization => width = {width}')
    for i, (start, end) in enumerate(bins):
        if i == bin_num - 1:
            print(f'bin{i + 1}: {start} < x <= {end}')
            break
        if i == 0:
            print(f'bin{i + 1}: {start} <= x <= {end}')
        else:
            print(f'bin{i + 1}: {start} < x <= {end}')
    print("=========================================================")
    bin_result = []
    for value in att_value:
        value = float(value)
        for i, (start, end) in enumerate(bins):
            if i == 0 and value >= start and value <= end:
                bin_result.append(i + 1)
                break
            elif i == bin_num - 1 and value > start and value <= end:
                bin_result.append(i + 1)
                break
            elif value > start and value <= end:
                bin_result.append(i + 1)
                break
    return bin_result

def discretize_all_features(df, features, bin_num):
    new_df = []
    # 對每個 feature 做離散化，回傳 bin 結果（list）
    bin_results = {}
    for feature in features:
        bin_results[feature] = equal_width(df, feature, bin_num)
    bin_results['class'] = df['class']
    return bin_results

dicretization_df = discretize_all_features(df, X, 10)
dicretization_df 

RI with equal width discretization => width = 0.0022780000000000022
bin1: 1.51115 <= x <= 1.513428
bin2: 1.513428 < x <= 1.515706
bin3: 1.515706 < x <= 1.517984
bin4: 1.517984 < x <= 1.520262
bin5: 1.520262 < x <= 1.52254
bin6: 1.52254 < x <= 1.524818
bin7: 1.524818 < x <= 1.527096
bin8: 1.527096 < x <= 1.529374
bin9: 1.529374 < x <= 1.531652
bin10: 1.531652 < x <= 1.53393
Na with equal width discretization => width = 0.6649999999999998
bin1: 10.73 <= x <= 11.395
bin2: 11.395 < x <= 12.059999999999999
bin3: 12.06 < x <= 12.725
bin4: 12.725 < x <= 13.389999999999999
bin5: 13.39 < x <= 14.055
bin6: 14.055 < x <= 14.719999999999999
bin7: 14.719999999999999 < x <= 15.384999999999998
bin8: 15.384999999999998 < x <= 16.049999999999997
bin9: 16.049999999999997 < x <= 16.714999999999996
bin10: 16.715 < x <= 17.38
Mg with equal width discretization => width = 0.449
bin1: 0.0 <= x <= 0.449
bin2: 0.449 < x <= 0.898
bin3: 0.898 < x <= 1.347
bin4: 1.347 < x <= 1.796
bin5: 1.796 < x <= 2.245
bin6: 2

{'RI': [5,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  5,
  4,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  4,
  3,
  5,
  5,
  3,
  3,
  3,
  5,
  3,
  4,
  4,
  7,
  5,
  4,
  6,
  4,
  4,
  4,
  3,
  3,
  1,
  4,
  3,
  3,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  6,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  2,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  4,
  4,
  3,
  3,
  4,
  3,
  3,
  4,
  8,
  6,
  6,
  9,
  10,
  5,
  4,
  7,
  8,
  8,
  4,
  4,
  4,
  4,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  5,
  4,
  3,
  5,
  5,
  4,
  5,
  7,
  4,
  4,
  4,
  3,
  4,
  3,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  5,
  3,
  3,
  3,
  3,
  3,
  5,
  3,
  3,
  4,
  4,
  5,
  2,
  4,
  5,
  5,
  4,
  3,
  4,
  6,
  1,
  1,
  5,
  5,
  5,
  4,
  4,
  4,
  4,
  1,
  4,
  4,
  4,
  1,
  1,
  4,
  6,
  5,
  6,
  3,
  3,
  3,
  3,
  3,
  2,
  2,
  3,
  2

In [133]:
forward_selection(dicretization_df, X, 'class')
backward_selection(dicretization_df, X, 'class')

[0.06074766355140187, 0.1822429906542056, 0.06074766355140187, 0.004672897196261682, 0.004672897196261682, 0.009345794392523364, 0.004672897196261682, 0.18691588785046728, 0.09345794392523364, 0.009345794392523364, 0.014018691588785047, 0.009345794392523364, 0.004672897196261682, 0.004672897196261682, 0.02336448598130841, 0.009345794392523364, 0.056074766355140186, 0.014018691588785047, 0.009345794392523364, 0.004672897196261682, 0.014018691588785047, 0.02336448598130841, 0.004672897196261682, 0.004672897196261682, 0.009345794392523364, 0.03271028037383177, 0.009345794392523364, 0.004672897196261682, 0.009345794392523364, 0.009345794392523364, 0.009345794392523364, 0.0794392523364486, 0.02336448598130841]
[0.1308411214953271, 0.5093457943925234, 0.21962616822429906, 0.04205607476635514, 0.014018691588785047, 0.03271028037383177, 0.028037383177570093, 0.014018691588785047, 0.004672897196261682, 0.004672897196261682]
[0.08450704225352113, 0.17370892018779344, 0.04225352112676056, 0.01877

(['Na', 'Mg', 'Al', 'Ca', 'Ba'], 0.41877684363493245)

### Equal Frequency

In [155]:
def equal_frequency(df,feature,bin_num):
    # 計算每個區間應包含的 instances 個數
    
    # 先記錄每個 instance 的原始索引及 value
    att_value = [(i, float(df[feature][i])) for i in range(len(df[feature]))]
    att_value.sort(key=lambda x: x[1])
    frequency = len(att_value) // bin_num
    bins = [] # 儲存每個 bin 範圍
    start = att_value[0][1]
    bin_index = 1    # 記錄目前 bin 
    cur_bin_cnt = 0  # 記錄目前 bin 所分配到的 value 個數
    # att_value 已排序過，故會由小到大遍歷
    for i,(org_index,value) in enumerate(att_value):
        # 目前 bin 的 value 數量超過一個 bin 所應該分配到的 frequency
        if cur_bin_cnt >= frequency and bin_index <= bin_num:
            # 且當前 value 不等於前一個 value 值
            if i < len(att_value) and value != att_value[i - 1][1]:
                if bin_index == bin_num:
                    bins.append((start,att_value[-1][1]))
                    break
                end = value  # 該 bin 的區間最大值
                bins.append((start,end))

                # 切換到下一個 bin
                bin_index += 1
                cur_bin_cnt = 0
                start = att_value[i][1]  # att_value[i][1] 為下個 bin 的起點
        # 該 bin 裡的 instances 個數加一
        cur_bin_cnt += 1

    # 上述設定在切換下個 bin 時才將 (start,end) 進 bins
    # 有可能迴圈結束，最後一個 bin 的值個數不足一個 frequency，不會切換 bin，因此需要額外判斷防止最後一個 bin 消失
    if len(bins) < bin_num:
        end = att_value[-1][1]   # end 為最後一個元素(最大值)
        bins.append((start, end))
    print(f'{feature} with equal frequency discretization:')
    for i, (start, end) in enumerate(bins):
        if i == bin_num - 1:
            print(f'bin{i + 1}: {start} < x <= {end}')
            break
        if i == 0:
            print(f'bin{i + 1}: {start} <= x <= {end}')
        else:
            print(f'bin{i + 1}: {start} < x <= {end}')
    print("=========================================================")
    org_value = df[feature]
    print(org_value)
    bin_result = []
    for value in org_value:
        value = float(value)
        for i, (start, end) in enumerate(bins):
            if i == 0 and value >= start and value <= end:
                bin_result.append(i + 1)
                break
            elif i == bin_num - 1 and value > start and value <= end:
                bin_result.append(i + 1)
                break
            elif value > start and value <= end:
                bin_result.append(i + 1)
                break
    return bin_result


def discretize_(df, features, bin_num):
    new_df = []
    # 對每個 feature 做離散化，回傳 bin 結果（list）
    bin_results = {}
    for feature in features:
        bin_results[feature] = equal_frequency(df, feature, bin_num)
    bin_results['class'] = df['class']
    return bin_results


dd = discretize_(df,X,10)


RI with equal frequency discretization:
bin1: 1.51115 <= x <= 1.51592
bin2: 1.51592 < x <= 1.51631
bin3: 1.51631 < x <= 1.5167
bin4: 1.5167 < x <= 1.51735
bin5: 1.51735 < x <= 1.51768
bin6: 1.51768 < x <= 1.51811
bin7: 1.51811 < x <= 1.5186
bin8: 1.5186 < x <= 1.51994
bin9: 1.51994 < x <= 1.52196
bin10: 1.52196 < x <= 1.53393
[1.52101, 1.51761, 1.51618, 1.51766, 1.51742, 1.51596, 1.51743, 1.51756, 1.51918, 1.51755, 1.51571, 1.51763, 1.51589, 1.51748, 1.51763, 1.51761, 1.51784, 1.52196, 1.51911, 1.51735, 1.5175, 1.51966, 1.51736, 1.51751, 1.5172, 1.51764, 1.51793, 1.51721, 1.51768, 1.51784, 1.51768, 1.51747, 1.51775, 1.51753, 1.51783, 1.51567, 1.51909, 1.51797, 1.52213, 1.52213, 1.51793, 1.51755, 1.51779, 1.5221, 1.51786, 1.519, 1.51869, 1.52667, 1.52223, 1.51898, 1.5232, 1.51926, 1.51808, 1.51837, 1.51778, 1.51769, 1.51215, 1.51824, 1.51754, 1.51754, 1.51905, 1.51977, 1.52172, 1.52227, 1.52172, 1.52099, 1.52152, 1.52152, 1.52152, 1.523, 1.51574, 1.51848, 1.51593, 1.51631, 1.51596, 1.51

In [156]:
forward_selection(dd, X, 'class')
backward_selection(dd, X, 'class')

[0.037383177570093455, 0.09813084112149532, 0.009345794392523364, 0.04672897196261682, 0.018691588785046728, 0.056074766355140186, 0.014018691588785047, 0.037383177570093455, 0.009345794392523364, 0.037383177570093455, 0.06074766355140187, 0.0514018691588785, 0.04672897196261682, 0.004672897196261682, 0.0514018691588785, 0.02336448598130841, 0.04672897196261682, 0.009345794392523364, 0.02336448598130841, 0.018691588785046728, 0.009345794392523364, 0.02336448598130841, 0.009345794392523364, 0.004672897196261682, 0.004672897196261682, 0.004672897196261682, 0.004672897196261682, 0.014018691588785047, 0.014018691588785047, 0.02336448598130841, 0.004672897196261682, 0.004672897196261682, 0.02336448598130841, 0.009345794392523364, 0.009345794392523364, 0.028037383177570093, 0.009345794392523364, 0.014018691588785047, 0.028037383177570093, 0.028037383177570093, 0.02336448598130841, 0.004672897196261682]
[0.09813084112149532, 0.102803738317757, 0.09813084112149532, 0.09813084112149532, 0.10747

(['RI', 'Na', 'Mg', 'Al', 'K', 'Ba'], 0.39072835610129586)

### Entropy Based

In [88]:
def entropy(df,class_label):
    class_value = [row[class_label] for row in df]
    counter = Counter(class_value)
    prob = [count / len(df) for i,count in counter.items()]
    entropy = -sum(p * log2(p) for p in prob) 
    return entropy

# cut_index 為資料要切成兩半時，「右側區間的起始 index」，此函式計算切割成左右 subset 後的資訊增益
def info_gain(df,cut_point,class_label):
    class_value = [row[class_label] for row in df]
    total = len(class_value)
    left = df[:cut_point]
    right = df[cut_point:]
    Ent_T = len(left) / total * entropy(left,class_label) + len(right) / total * entropy(right,class_label)

    info_gain = entropy(df,class_label) - Ent_T
    return info_gain


def find_cut_point(df, feature, class_label):
    best_info_gain = -1
    best_cut_index = -1
    best_cut_value = None
    # 針對選定的 attribute 依照 attribute value 由小到大排序
    sorted_df = sorted(df, key = lambda row : float(row[feature]))
    
    for i in range(1, len(sorted_df)):
        if sorted_df[i][class_label] != sorted_df[i - 1][class_label]:
            cut_value = (float(sorted_df[i][feature]) + float(sorted_df[i - 1][feature])) / 2
            cur_gain = info_gain(df,i,class_label)
            if cur_gain > best_info_gain:
                best_info_gain = cur_gain
                best_cut_index = i
                best_cut_value = cut_value
    return best_info_gain, best_cut_index, best_cut_value

def split(df,feature,class_label,cut_points):
    best_info_gain, best_cut_index, best_cut_value = find_cut_point(df,feature,class_label)
    class_value = [row[class_label] for row in df]
    left_set = [row for row in df if float(row[feature]) <= best_cut_value]
    right_set = [row for row in df if float(row[feature]) > best_cut_value]
    left_class = [row[class_label] for row in left_set]
    right_class = [row[class_label] for row in right_set]
   
    left_cnt = Counter(left_class)
    right_cnt = Counter(right_class)
    total_cnt = Counter(class_value)
    k = len(total_cnt)
    k1 = len(left_cnt)
    k2 = len(right_cnt)

    print( entropy(left_set,class_label) )
    delta = log2(3 ** k - 2) - (k * entropy(df,class_label) 
            - k1 * entropy(left_set,class_label)
            - k2 * entropy(right_set,class_label))
    print(delta)
    gain_threshold = log2(len(df) - 1) / len(df) + delta
    print(gain_threshold)
    print(best_info_gain)
    if best_info_gain <= gain_threshold or len(df) <= 1:
        return
    

    if len(left_set) == 0 or len(right_set) == 0:
        return
    cut_points.append(best_cut_value)
  
    split(left_set,feature,class_label,cut_points)
    split(right_set,feature,class_label,cut_points)
    

def recursive_ent_base(df,feature,class_label,gain_threshold):
    cut_points = []
    split(df,feature,class_label,cut_points)
    return sorted(cut_points)

In [89]:
cut_points = recursive_ent_base(df,"Mg",'class',0.6)
cut_points

2.3234252671078384
14.397420467643926
14.433563970542178
0.9019255901324292


[]